In [1]:
val WINDOW = 32;
val STEP = 2;
val SAMPLES = 200000;
val scale = 1/200.0;  // Mv


In [2]:
// Databricks  
//val rootDir = "FileStore/fxo/ted/"   //local File


Name: Syntax Error.
Message: 
StackTrace: 

In [3]:
// GCP
val rootDir = "hdfs://spark-clu-m/user/fsainz/data/out/" 

In [4]:

import org.apache.spark.mllib.clustering.{KMeans, KMeansModel}
import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.mllib.regression.LabeledPoint 



In [5]:
var windowingF  =  new Array[Double](WINDOW)   //differs of Array[Double](WINDOW) uajj
for ( i <- 0 until WINDOW) {
    val y = Math.sin(Math.PI * i / (WINDOW - 1.0));
          windowingF(i)=  y * y
 }



In [6]:
def processDataframe ( wSize: Int, frame : Array[Double] , model:KMeansModel) : Array[Double] = {
  
  // Split frame in WINDOW size windows
  val frameLength = frame.length 
  
  assert( frameLength % wSize == 0 )  // should throws a exception if not
  
  val signalWindows = scala.collection.mutable.ArrayBuffer.empty[Array[Double]]
  val codecWindows = scala.collection.mutable.ArrayBuffer.empty[Array[Double]]
  val ret = scala.collection.mutable.ArrayBuffer.empty[Double]
  val nWindows:Int = frameLength / wSize 
  
  for ( i <- 0 until (nWindows * 2)-1) {   // we step 1/2 wSize
    var p = i*wSize/2
    var signalWindow = frame.slice(p, p+wSize  )
   
    var tapWindow = (signalWindow,windowingF).zipped.map(_*_)   // multiple vector item by item
    val wNorm = Vectors.norm(Vectors.dense(tapWindow) , 2)
    tapWindow = tapWindow.map( x => x/wNorm)   
   
    val nc = model.predict(Vectors.dense(tapWindow))
    
    val cluster = model.clusterCenters(nc)
   
    val codecWindow = cluster.toArray.map(x => x*wNorm)   // scale to signal 
 
    signalWindows.append(signalWindow)
    codecWindows.append(codecWindow.toArray)
    // Coded windows has tapering function scale. get it of it usign a trick sum two desplacements halfs. 
    // see propierties or tapering funtion.
    // we produce half a window a time
    // and we have to arrange in some way first and last halfs
    var half1 = new Array[Double](wSize)
    var half2 = new Array[Double](wSize)
    if (i == 0) {
      half1 = codecWindows(i).slice(0,wSize/2)  
      half2 = half1  
    } 
    else {
      print(","+i+",")
      half1= codecWindows(i-1).slice(wSize/2,wSize)
      half2= codecWindows(i).slice(0,wSize/2)  
      
    }
    var r = (half1 , half2).zipped.map(_+_)   // sum halfs
    ret ++= r
    // last half window 
    if ( i == (nWindows*2) -2 ) {
      half1= codecWindows(i).slice(wSize/2,wSize)
      half2= half1 
      print ("*************LAST + " + i)
      r = (half1 , half2).zipped.map(_+_)   // sum halfs
      ret ++= r
    }
  }
  ret.toArray
}

In [7]:

val csvFileName = rootDir + "frames-a02.csv"
val framesECGRdd = sc.textFile(csvFileName).map(x => x.split(",").map(_.toDouble) )

In [9]:

val clusters = KMeansModel.load(sc, rootDir + "clusters-a02.model")

In [10]:
val framesCodedRdd = framesECGRdd.map( x => (x, processDataframe(WINDOW, x ,clusters ) )).map(x => (x._1, x._2, (x._1,x._2).zipped.map(_-_)))

In [33]:
framesCodedRdd.first._2.length

In [24]:
val sqlContext = new SQLContext(sc)
import org.apache.spark.sql.SQLContext
import sqlContext.implicits._
framesCodedRdd.toDF.createOrReplaceTempView("framesProccesed")

In [13]:
framesCodedRdd.first._2

Array(0.0, -0.17033731469472463, -0.717587255486073, -1.6443707239788117, -3.008325681765548, -4.672696373233639, -6.937489663316098, -8.728333664575432, -11.735259518263517, -13.69173554456661, -17.052369410925444, -18.789221689858927, -21.753121458052835, -22.865286305072438, -25.085968785164773, -20.899083871184242, -3.3394402666518817, 4.4836301822035205, 6.5048418471739495, 13.870409202543007, 27.284385923510396, 20.58716473187961, 7.7384088268779205, 0.39773206602598077, -4.706234479185415, -13.022765772724476, -24.93486483477633, -29.88374242851996, -29.763253341886966, -26.658623825549466, -31.459493344629088, -20.005565988772084, 29.551227070244536, -52.670004318776904, 57.81694307060094, 72.64351910129903, -86.33007994630364, -75.12524819389554, -42.484...

In [ ]:
val sqlContext = new SQLContext(sc)
import org.apache.spark.sql.SQLContext
import sqlContext.implicits._



In [27]:
val framesCodedDF = framesCodedRdd.toDF
framesCodedDF.createOrReplaceTempView("framesProccesed")

In [32]:
var outputFileName ="/user/fsainz/data/out/framesCoded-a02.csv"
framesCodedDF.write.format("com.databricks.spark.csv").option("header", "false").mode("overwrite").save(outputFileName)

Name: java.lang.UnsupportedOperationException
Message: CSV data source does not support array<double> data type.
StackTrace:   at org.apache.spark.sql.execution.datasources.csv.CSVUtils$.org$apache$spark$sql$execution$datasources$csv$CSVUtils$$verifyType$1(CSVUtils.scala:127)
  at org.apache.spark.sql.execution.datasources.csv.CSVUtils$$anonfun$verifySchema$1.apply(CSVUtils.scala:131)
  at org.apache.spark.sql.execution.datasources.csv.CSVUtils$$anonfun$verifySchema$1.apply(CSVUtils.scala:131)
  at scala.collection.Iterator$class.foreach(Iterator.scala:893)
  at scala.collection.AbstractIterator.foreach(Iterator.scala:1336)
  at scala.collection.IterableLike$class.foreach(IterableLike.scala:72)
  at org.apache.spark.sql.types.StructType.foreach(StructType.scala:98)
  at org.apache.spark.sql.execution.datasources.csv.CSVUtils$.verifySchema(CSVUtils.scala:131)
  at org.apache.spark.sql.execution.datasources.csv.CSVFileFormat.prepareWrite(CSVFileFormat.scala:65)
  at org.apache.spark.sql.

In [31]:

framesCodedDF.createOrReplaceTempView("framesProccesed")